In [ ]:
#All code adapted from: https://www.kaggle.com/code/tard232003/20bd1a6754-basic-shapes
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from matplotlib import image
import os

# Input data files are available in the read-only "../input/" directory

for dirname, _, filenames in os.walk('shapes'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
im=image.imread('shapes/squares/drawing(1).png')


In [ ]:
plt.imshow(im)

In [ ]:

#Flattening the images, makes each image into 1d-array -> with the output
from PIL import Image
def load_imgs(impath):
    imgs=[]
    label=[]
    l1=os.listdir(impath)
    for i in l1:
        l2=os.listdir(impath+'/'+i)
        for j in l2:
            c=0
            img=Image.open(impath+i+'/'+j)
            img=img.resize(size=(32,32))
            img=img.convert('L')
            imgs.append(np.array(img).flatten())
            label.append(i)
            del img
    return np.array(imgs),label

In [ ]:
#getting flattened images, each shape( triangle, circle, square) has 100 images, and each image is an array (32 x 32 = 1024 elements), 
# and the output y is the shape each array is associated with.
x,y=load_imgs('shapes/shapes/')
x.shape,len(y)

In [ ]:
# Printing possible truth - labels
target=pd.Series(y,dtype='category')
target

In [ ]:
t=target.cat.codes
t

In [ ]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(x,t,test_size=0.2,shuffle=True)
train_x.shape,test_x.shape,train_y.shape,test_y.shape

In [ ]:
#LOGISTIC REGRESSION model
from sklearn.linear_model import LogisticRegression
lg=LogisticRegression(max_iter=200,solver='saga')
lg.fit(train_x,train_y)

In [ ]:
yhat=lg.predict(test_x)   #making predictions, but modle might need more training, has no validation phase
yhat

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y,yhat)   #computing accuarcy

In [ ]:
#SVC model
from sklearn.svm import SVC 
lg=SVC()
lg.fit(train_x,train_y)

In [ ]:
yhat=lg.predict(test_x)
yhat

In [ ]:
accuracy_score(test_y,yhat)

In [ ]:
#Sequential Model (neural network)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model=Sequential()
model.add(Dense(64,input_dim=train_x.shape[1],activation='relu',name='sh_dense1'))
model.add(Dense(3,activation='softmax',name='sh_dense2'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()
model.fit(train_x,train_y,epochs=300)

In [ ]:
z=model.evaluate(train_x,train_y)   #will have to change this so that model does not use the same data as training in the evaluation phase

In [ ]:
z=model.evaluate(test_x,test_y) # testing